In [1]:
# importing all libraries
import pandas as pd
import numpy as np
from sklearn.datasets import load_diabetes

In [2]:
X,Y = load_diabetes(return_X_y=True)

In [3]:
X.shape

(442, 10)

In [4]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=2)

# Sklearn LR

In [5]:
from sklearn.linear_model import LinearRegression
lr=LinearRegression()

In [6]:
lr.fit(X_train,Y_train)

LinearRegression()

In [7]:
lr.intercept_

151.88331005254167

In [8]:
lr.coef_

array([  -9.15865318, -205.45432163,  516.69374454,  340.61999905,
       -895.5520019 ,  561.22067904,  153.89310954,  126.73139688,
        861.12700152,   52.42112238])

In [9]:
Y_pred=lr.predict(X_test)

In [10]:
from sklearn.metrics import r2_score
r2_score(Y_test,Y_pred)

0.4399338661568968

In [11]:
X_train.shape

(353, 10)

# My Batch GD Regressor class

In [12]:
class GDRegressor:
    def __init__(self,lr=0.01,epochs=100):
#       initializing m and b to none
        self.coef=None
        self.intercept=None
        self.lr=lr
        self.epochs=epochs
        
    def fit(self,X_train,Y_train):
#       assuming all coeff values to 1 and intercept value to 0
        self.coef=np.ones(X_train.shape[1])
        self.intercept=0
        
#       for a particular range we update coeff and intercept value
#       beta_new = beta_old - (lr*slope)
        for i in range(self.epochs):
            Y_hat= np.dot(X_train,self.coef)+self.intercept #  Y = mX + b
            intercept_der=-2 * np.mean(Y_train-Y_hat) # (-2/n) * (sum) (Y- Y_pred)^2
            self.intercept=self.intercept - (self.lr*intercept_der)
            
            coef_der=-2*np.dot((Y_train-Y_hat),X_train)/X_train.shape[0] # (-2) * ((sum) (Y- Y_pred)^2 * X )/ n
            self.coef=self.coef - (self.lr*coef_der)
            
        print(self.intercept,self.coef)
    def predict(self,X_test):
        return np.dot(X_test,self.coef)+self.intercept #  Y = mX + b

In [13]:
gdr=GDRegressor(lr=0.5,epochs=1000)

In [14]:
gdr.fit(X_train,Y_train)

152.01351687661833 [  14.38990585 -173.7235727   491.54898524  323.91524824  -39.32648042
 -116.01061213 -194.04077415  103.38135565  451.63448787   97.57218278]


In [15]:
Y_pred=gdr.predict(X_test)

In [16]:
r2_score(Y_test,Y_pred)

0.4534503034722803

# My Stochastic GD Regressor Class

In [17]:
class SGDRegressor:
#  in batch stochastic GD we update the value of m and b in every iteration for whole dataset but here we update for a random row input
#  it is faster than Batch GD but not for same no of epochs
    def __init__(self,lr=0.01,epochs=100):
        self.coef=None
        self.intercept=None
        self.lr=lr
        self.epochs=epochs
        
    def fit(self,X_train,Y_train):
        self.coef=np.ones(X_train.shape[1])
        self.intercept=0
        
        for i in range(self.epochs):
            for j in range(X_train.shape[0]):
                
                ind=np.random.randint(0,X_train.shape[0]) # calculating a random row and same operations
                Y_hat= np.dot(X_train[ind],self.coef)+self.intercept
                intercept_der=-2 * np.mean(Y_train[ind]-Y_hat)
                self.intercept=self.intercept - (self.lr*intercept_der)
            
                coef_der=-2*np.dot((Y_train[ind]-Y_hat),X_train[ind])
                self.coef=self.coef - (self.lr*coef_der)
            
        print(self.intercept,self.coef)
    def predict(self,X_test):
        return np.dot(X_test,self.coef)+self.intercept

In [18]:
sgd=SGDRegressor(lr=0.01,epochs=40)

In [19]:
sgd.fit(X_train,Y_train)

149.58404296757186 [  65.56359888  -51.92474897  323.77361636  232.13948179   29.56421954
   -8.72102505 -158.15816072  124.40472402  285.73784952  126.58066595]


In [20]:
Y_pred=sgd.predict(X_test)

In [21]:
r2_score(Y_test,Y_pred)

0.42058282414502135

# My MiniBatch GD Regressor Class

In [22]:
import random

#  here we divide whole dataset into certain batch and then update batch wise
class MGDRegressor:
    def __init__(self,batch_size,lr=0.01,epochs=100):
        self.coef=None
        self.intercept=None
        self.lr=lr
        self.epochs=epochs
        self.batch_size=batch_size
        
    def fit(self,X_train,Y_train):
        self.coef=np.ones(X_train.shape[1])
        self.intercept=0
        
        for i in range(self.epochs):
            for j in range(int(X_train.shape[0]/self.batch_size)):
                
                ind=random.sample(range(X_train.shape[0]),self.batch_size)
                Y_hat= np.dot(X_train[ind],self.coef)+self.intercept
                intercept_der=-2 * np.mean(Y_train[ind]-Y_hat)
                self.intercept=self.intercept - (self.lr*intercept_der)
            
                coef_der=-2*np.dot((Y_train[ind]-Y_hat),X_train[ind])
                self.coef=self.coef - (self.lr*coef_der)
            
        print(self.intercept,self.coef)
    def predict(self,X_test):
        return np.dot(X_test,self.coef)+self.intercept

In [23]:
mgd=MGDRegressor(batch_size=int(X_train.shape[0]/50),lr=0.01,epochs=100)

In [24]:
mgd.fit(X_train,Y_train)

150.79569678313376 [  25.18729286 -132.29794932  452.81830601  301.15095907  -28.67216565
  -94.83105107 -197.29760482  116.55746024  407.71959096  117.01195885]


In [25]:
Y_pred=mgd.predict(X_test)

In [26]:
r2_score(Y_test,Y_pred)

0.45299646698279117